In [1]:
# ========= 0) Fresh install for Colab (CUDA 12.4 stack) =========
!pip -q uninstall -y torch torchvision torchaudio >/dev/null
!pip -q install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip -q install -U transformers sentence-transformers rank-bm25 pandas tqdm
# Optional helpers
!pip -q install -U hf_transfer  # faster HF downloads
# !pip -q install -U faiss-cpu   # optional: for huge corpora (not needed here)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.5/768.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os, re, unicodedata, time, json, math
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
# ---- quiet + fast HF downloads
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def device_auto():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

DEV = device_auto()
print("Device:", DEV)

Device: cuda


In [4]:
# ---- knobs you can tweak easily
FAST_MODE = True                      # False = highest quality (slower)
SHORTLIST_K = 10 if FAST_MODE else 20
RERANK_MODEL = "BAAI/bge-reranker-base" if FAST_MODE else "BAAI/bge-reranker-v2-m3"
RERANK_BATCH = 8 if FAST_MODE else 16
DENSE_BATCH = 64 if torch.cuda.is_available() else 32  # embeddings

In [5]:
# ========= 1) Data (your original list) =========
data = {
  "books": [
    {"id":"mm001","name":"မဟာရာဇာဝင်","author":"ဦးကလျာ","description":"မြန်မာ့ရှေးဟောင်း ဘုရင်များ၏ သမိုင်းကြောင်းနှင့် ဓလေ့ထုံးတမ်းများအကြောင်း ဖော်ပြထားသော စာအုပ်","category":"သမိုင်း","tags":["သမိုင်း","ဘုရင်","ဓလေ့ထုံးတမ်း","ရှေးဟောင်း","မြန်မာ့ယဉ်ကျေးမှု"]},
    {"id":"mm002","name":"ထွန်းကားသော ရွှေဂုဏ်ရည်","author":"စိန်တင်အောင်","description":"မြန်မာ့လွတ်လပ်ရေးတော်လှန်ရေးကာလ ရဲဘော်များ၏ စိတ်ဓာတ်နှင့် စေတနာအကြောင်း ရေးသားထားသော ဝတ္ထု","category":"ဝတ္ထု","tags":["ဝတ္ထု","လွတ်လပ်ရေး","တော်လှန်ရေး","ရဲဘော်","စေတနာ"]},
    {"id":"mm003","name":"ကုလားမြေ ဧရိယာများ","author":"မသွေး","description":"ကုလားမြေဒေသ၏ ရိုးရာယဉ်ကျေးမှုနှင့် လူမှုဘဝများကို ဖော်ကျူးထားသော စာပေလက်ရာ","category":"စာပေ","tags":["စာပေ","ကုလားမြေ","ရိုးရာယဉ်ကျေးမှု","လူမှုဘဝ","ဒေသ"]},
    {"id":"mm004","name":"ရန်ကုန်မြို့ မုန်းညှင်းကွင်း","author":"အလွင်မောင်","description":"ခေတ်မီရန်ကုန်မြို့တွင် နေထိုင်သော လူငယ်များ၏ အချစ်ဇာတ်လမ်းနှင့် ဘဝရုန်းကန်မှုများ","category":"ခေတ်သစ်ဝတ္ထု","tags":["ခေတ်သစ်","အချစ်","လူငယ်","ရန်ကုန်","ရုန်းကန်မှု"]},
    {"id":"mm005","name":"မိုးရာသီ နောက်လိုက်","author":"မဆန်း","description":"မြန်မာ့လယ်ကွင်းဒေသ လယ်သမားများ၏ ဘဝကြမ်းတမ်းမှုနှင့် သဘာဝဒဏ်ရာများကို ရင်ဆိုင်ပုံ","category":"လူမှုရေးဝတ္ထု","tags":["လယ်ကွင်း","လယ်သမား","သဘာဝဒဏ်ရာ","လူမှုရေး","မုန်းတိုင်း"]},
    {"id":"mm006","name":"တက္ကသိုလ်နေ့များ","author":"ဇော်ဂျီ","description":"တက္ကသိုလ်ကျောင်းသား၊ကျောင်းသူများ၏ ပညာရေးဘဝနှင့် အနာဂတ်အိပ်မက်များ","category":"လူငယ်စာပေ","tags":["တက္ကသိုလ်","ကျောင်းသား","ပညာရေး","အိပ်မက်","လူငယ်"]},
    {"id":"mm007","name":"မြန်မာ့ကဗျာ ရွေးချယ်စု","author":"မင်းသားမြင့်","description":"မြန်မာ့ခေတ်သစ်ကဗျာများမှ ရွေးချယ်ပြီး စုစည်းထားသော ကဗျာအစုံ","category":"ကဗျာ","tags":["ကဗျာ","ခေတ်သစ်","စုစုံ","ရွေးချယ်","အနုပညာ"]},
    {"id":"mm008","name":"ရွှေမိန်းကလေး ဂီတကမ္ဘာ","author":"စိုးမြင့်နိုင်","description":"မြန်မာ့ရိုးရာဂီတနှင့် ခေတ်မီဂီတ၏ ပေါင်းစပ်မှုအကြောင်း ရေးသားထားသော စာအုပ်","category":"ဂီတ","tags":["ဂီတ","ရိုးရာ","ခေတ်မီ","အနုပညာ","ယဉ်ကျေးမှု"]},
    {"id":"mm009","name":"ဗုဒ္ဓဓမ္မ လမ်းညွှန်","author":"သီတဂူဆရာတော်","description":"ဗုဒ္ဓဘာသာ၏ အခြေခံတရားများနှင့် ဘဝတွင်အသုံးချနိုင်သော ဓမ္မဒေသနာများ","category":"ဘာသာရေး","tags":["ဗုဒ္ဓဘာသာ","ဓမ္မ","တရား","ဘဝ","လမ်းညွှန်"]},
    {"id":"mm010","name":"မေတ္တာ သုခုမလေး","author":"မောင်မောင်လွင်","description":"မေတ္တာ၊ ကရုဏာ၊ မုဒိတာ၊ ဥပေက္ခာ ဗြဟ္မဝိഹာရ လေးပါးအကြောင်း လွယ်ကူစွာ နားလည်နိုင်သော ရေးနည်း","category":"ဘာသာရေး","tags":["မေတ္တာ","ကရုဏာ","ဗြဟ္မဝိဟာရ","ဓမ္မ","သုခုမ"]},
    {"id":"mm011","name":"ခေတ်မီသိပ္ပံနည်းကျ စာကြည့်တိုက်","author":"ဒေါက်တာခင်မြင့်","description":"ခေတ်မီသိပ္ပံနည်းပညာများကို မြန်မာဘာသာဖြင့် လွယ်ကူစွာ နားလည်နိုင်အောင် ရေးသားထားသော စာအုပ်","category":"သိပ္ပံ","tags":["သိပ္ပံ","နည်းပညာ","ခေတ်မီ","ပညာရေး","သင်ခန်းစာ"]},
    {"id":"mm012","name":"မြန်မာ့သုခုမငယ် ခံ့မြတ်လမ်းစဉ်","author":"ဦးချမ်းမြေ့","description":"မြန်မာ့လူငယ်များအတွက် ကျန်းမာရေးထိန်းသိမ်းခြင်းနှင့် စိတ်ဓာတ်ခံ့မြတ်စေခြင်း အကြောင်း","category":"ကျန်းမာရေး","tags":["ကျန်းမာရေး","လူငယ်","စိတ်ဓာတ်","ခံ့မြတ်","လမ်းညွှန်"]},
    {"id":"mm013","name":"ကမ္ပျူတာသုံး မြန်မာစာ","author":"မြင့်စိုး","description":"ကမ္ပျူတာတွင် မြန်မာစာ ရိုက်နှိပ်ခြင်းနှင့် အသုံးပြုခြင်းအကြောင်း သင်ခန်းစာများ","category":"ကမ္ပျူတာ","tags":["ကမ္ပျူတာ","မြန်မာစာ","နည်းပညာ","သင်ခန်းစာ","ရိုက်နှိပ်"]},
    {"id":"mm014","name":"အောင်ဆန်းနှင့် လွတ်လပ်ရေး","author":"ဒေါက်တာသန်းတုန်း","description":"ဗိုလ်ချုပ်အောင်ဆန်း၏ ဘဝတွင်းနှင့် မြန်မာ့လွတ်လပ်ရေးရရှိမှုတွင် ပါဝင်ကူညီခဲ့သော အခန်းကဏ္ဍများ","category":"သမိုင်းဝင်ပုဂ္ဂိုလ်","tags":["အောင်ဆန်း","လွတ်လပ်ရေး","သမိုင်းဝင်","ပုဂ္ဂိုလ်","မြန်မာ"]},
    {"id":"mm015","name":"ပုဂံခေတ် ကျောက်စာများ","author":"ဦးပေးမောင်တင်","description":"ပုဂံခေတ်မှ ရေးသားထားသော ကျောက်စာများနှင့် ၄င်းတို့မှ ရရှိသော သမိုင်းအချက်အလက်များ","category":"ရှေးပုရာဏ်","tags":["ပုဂံ","ကျောက်စာ","ရှေးပုရာဏ်","သမိုင်း","ခေတ်ဟောင်း"]},
    {"id":"mm016","name":"မြန်မာ့စီးပွားရေး ခေတ်စားလမ်း","author":"ဦးအောင်မြင့်","description":"မြန်မာနိုင်ငံ၏ စီးပွားရေးအခြေအနေနှင့် ခေတ်မီစီးပွားရေး နည်းလမ်းများ အကြောင်း","category":"စီးပွားရေး","tags":["စီးပွားရေး","ခေတ်မီ","လမ်းညွှန်","မြန်မာ","စီးပွား"]},
    {"id":"mm017","name":"စုစုံကုန်ကျွေးမွေးရေး","author":"ဒေါ်မြသီရိ","description":"စုစုံကုန်လုပ်ငန်း စတင်ခြင်းမှ စာရင်းကိုင်ခြင်းအထိ လိုအပ်သော ဗဟုသုတများ","category":"လုပ်ငန်း","tags":["လုပ်ငန်း","စုစုံကုန်","ကုန်ကွန်း","စာရင်းကိုင်","မွေးမြူရေး"]},
    {"id":"mm018","name":"သုံးပေါင်းရဲဘော်","author":"အန်တီမြသန်း","description":"ကလေးသုံးပေါင်းအတွက် ရေးသားထားသော စွန့်စားခန်းနှင့် မိတ်ဆွေသစ်များ ရှာဖွေခြင်း ဇာတ်လမ်း","category":"ကလေးစာပေ","tags":["ကလေး","စွန့်စား","မိတ်ဆွေ","ပေါင်းသင်း","ဇာတ်လမ်း"]},
    {"id":"mm019","name":"ညွန့်ညွန့်ကလေး နံနက်ပိုင်း","author":"ဦးဘဦး","description":"ကလေးငယ်များအတွက် နံနက်ခင်းမှ ညနေခင်းအထိ လုပ်ကိုင်သင့်သော အခြေခံကျသော အရာများ","category":"ကလေးပညာ","tags":["ကလေး","နံနက်ပိုင်း","အခြေခံ","ပညာ","လုပ်ကိုင်ပုံ"]},
    {"id":"mm020","name":"ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမှု","author":"ဒေါက်တာဝင်းလွင်","description":"ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမှုကြောင့် မြန်မာနိုင်ငံ ခံစားနေရသော ဆိုးကျိုးများနှင့် ကာကွယ်နိုင်သော နည်းလမ်းများ","category":"သဘာဝပတ်ဝန်းကျင်","tags":["ရာသီဥတု","ပတ်ဝန်းကျင်","ပြောင်းလဲမှု","သဘာဝ","ကာကွယ်ရေး"]},
    {"id":"mm021","name":"မြန်မာ့ရိုးရာ အစားအသောက်","author":"ဒေါ်ခင်မျိုးရီ","description":"မြန်မာနိုင်ငံ တစ်နိုင်ငံလုံးမှ ရိုးရာအစားအသောက်များနှင့် ၄င်းတို့ကို ပြုလုပ်နည်းများ","category":"ရိုးရာယဉ်ကျေးမှု","tags":["ရိုးရာ","အစားအသောက်","ယဉ်ကျေးမှု","ပြုလုပ်နည်း","အစဉ်အလာ"]},
    {"id":"eng001","name":"Digital Marketing Revolution","author":"Michael Chen","description":"Complete guide to modern digital marketing strategies and social media promotion techniques","category":"Business","tags":["marketing","digital","social media","business","strategy"]},
    {"id":"eng002","name":"Love in Yangon Streets","author":"Sarah Thompson","description":"A romantic novel set in modern Yangon exploring love, tradition, and cultural change","category":"Romance","tags":["romance","yangon","love","culture","modern"]},
    {"id":"mm022", "name": "မြန်မာ့သမိုင်းပုံပြင်များ", "author": "မောင်သိန်းနိုင်", "description": "မြန်မာ့ရှေးဟောင်းသမိုင်းမှ စိတ်ဝင်စားဖွယ် ပုံပြင်များနှင့် ဒဏ္ဍာရီများ", "category": "ပုံပြင်", "tags": ["သမိုင်း", "ပုံပြင်", "ဒဏ္ဍာရီ", "ရှေးဟောင်း", "မြန်မာ့ယဉ်ကျေးမှု"]},
    {"id":"mm023", "name": "ခေတ်သစ်ပန်းချီလက်ရာများ", "author": "ဒေါ်နုနု", "description": "မြန်မာ့ခေတ်သစ်ပန်းချီလောကမှ ထင်ရှားသော ပန်းချီကားများနှင့် ပန်းချီဆရာများအကြောင်း", "category": "အနုပညာ", "tags": ["အနုပညာ", "ပန်းချီ", "ခေတ်သစ်", "လက်ရာ", "မြန်မာ"]},
    {"id":"eng003", "name": "The Art of Negotiation", "author": "David Lee", "description": "Practical strategies and techniques for effective negotiation in business and life", "category": "Business", "tags": ["business", "negotiation", "strategy", "communication", "skills"]},
    {"id":"mm024", "name": "စိတ်ပညာ အခြေခံ", "author": "ဦးကျော်မြင့်", "description": "လူ့စိတ်အကြောင်းနှင့် အခြေခံစိတ်ပညာသဘောတရားများ", "category": "စိတ်ပညာ", "tags": ["စိတ်ပညာ", "လူ့စိတ်", "သဘောတရား", "ပညာရေး", "အခြေခံ"]},
    {"id":"mm025", "name": "မြန်မာ့ရိုးရာ တေးဂီတ", "author": "ဦးစိုးနိုင်", "description": "မြန်မာ့ရိုးရာတေးဂီတများ၏ သမိုင်းနှင့် တူရိယာများအကြောင်း", "category": "ဂီတ", "tags": ["ဂီတ", "ရိုးရာ", "တေးဂီတ", "သမိုင်း", "တူရိယာ"]},
    {"id":"mm026", "name": "ကလေးများအတွက် ပုံပြောနည်း", "author": "ဒေါ်ခင်မာလာ", "description": "ကလေးငယ်များ စိတ်ဝင်စားအောင် ပုံပြောနည်း အခြေခံများနှင့် နမူနာများ", "category": "ကလေးပညာ", "tags": ["ကလေး", "ပုံပြော", "ပညာရေး", "သင်ခန်းစာ", "အခြေခံ"]},
    {"id":"eng004", "name": "Introduction to Artificial Intelligence", "author": "Andrew Ng", "description": "Fundamental concepts and applications of AI and machine learning", "category": "Technology", "tags": ["AI", "machine learning", "technology", "education", "computer science"]},
    {"id":"mm027", "name": "မြန်မာ့ရိုးရာ ဗိသုကာလက်ရာများ", "author": "ဦးသောင်းမြင့်", "description": "မြန်မာ့ရှေးဟောင်း ဗိသုကာလက်ရာများ၏ အနုပညာနှင့် သမိုင်းတန်ဖိုး", "category": "အနုပညာ", "tags": ["အနုပညာ", "ဗိသုကာ", "ရှေးဟောင်း", "သမိုင်း", "မြန်မာ"]},
    {"id":"mm028", "name": "ကိုယ်ပိုင်လုပ်ငန်း စတင်ခြင်း", "author": "ဦးကျော်စွာ", "description": "လုပ်ငန်းအသစ် စတင်ရန် လိုအပ်သော အခြေခံဗဟုသုတများနှင့် စီမံခန့်ခွဲမှု နည်းလမ်းများ", "category": "လုပ်ငန်း", "tags": ["လုပ်ငန်း", "စီးပွားရေး", "စီမံခန့်ခွဲမှု", "အခြေခံ", "လမ်းညွှန်"]},
    {"id":"eng005", "name": "The Power of Habit", "author": "Charles Duhigg", "description": "Understanding how habits work and how to change them for personal and professional success", "category": "Self-Help", "tags": ["self-help", "habit", "psychology", "personal development", "success"]},
    {"id":"mm029", "name": "မြန်မာ့ဆေးဖက်ဝင်အပင်များ", "author": "ဒေါက်တာအေးအေးသန်း", "description": "မြန်မာနိုင်ငံတွင် တွေ့ရသော ဆေးဖက်ဝင်အပင်များနှင့် ၄င်းတို့၏ အကျိုးကျေးဇူးများ", "category": "ကျန်းမာရေး", "tags": ["ကျန်းမာရေး", "ဆေးဖက်ဝင်", "အပင်", "သဘာဝ", "တိုင်းရင်းဆေး"]},
    {"id":"mm030", "name": "ခရီးသွားမှတ်တမ်း - ပုဂံ", "author": "မောင်မောင်ဦး", "description": "ပုဂံဒေသသို့ ခရီးသွားမှတ်တမ်းနှင့် ပုဂံ၏ သမိုင်းဝင်နေရာများအကြောင်း", "category": "ခရီးသွား", "tags": ["ခရီးသွား", "ပုဂံ", "သမိုင်းဝင်", "နေရာ", "မှတ်တမ်း"]},
    {"id":"eng006", "name": "Data Science for Beginners", "author": "Joe James", "description": "An easy-to-understand introduction to data science concepts and tools", "category": "Technology", "tags": ["data science", "technology", "education", "programming", "analysis"]},
    {"id":"mm031", "name": "မြန်မာ့ရိုးရာ အားကစား", "author": "ဦးဘသိန်း", "description": "မြန်မာ့ရိုးရာ အားကစားနည်းများ၏ သမိုင်းနှင့် ကစားနည်းများ", "category": "အားကစား", "tags": ["အားကစား", "ရိုးရာ", "ကစားနည်း", "သမိုင်း", "မြန်မာ"]},
    {"id":"mm032", "name": "ပန်းဥယျာဉ်စိုက်ပျိုးနည်း", "author": "ဒေါ်ခင်မာ", "description": "အိမ်တွင်းနှင့် အိမ်ပြင် ပန်းဥယျာဉ်စိုက်ပျိုးနည်း အခြေခံများနှင့် လျှို့ဝှက်ချက်များ", "category": "စိုက်ပျိုးရေး", "tags": ["စိုက်ပျိုးရေး", "ပန်းဥယျာဉ်", "အပင်", "နည်းလမ်း", "အခြေခံ"]},
    {"id":"eng007", "name": "The Lean Startup", "author": "Eric Ries", "description": "How to build successful businesses using continuous innovation and validated learning", "category": "Business", "tags": ["business", "startup", "innovation", "entrepreneurship", "management"]},
     {"id":"mm033", "name": "မြန်မာ့ရိုးရာ ပွဲတော်များ", "author": "မောင်စိုး", "description": "မြန်မာနိုင်ငံအနှံ့အပြားမှ ထင်ရှားသော ရိုးရာပွဲတော်များ၏ သမိုင်းနှင့် ကျင်းပပုံများ", "category": "ရိုးရာယဉ်ကျေးမှု", "tags": ["ရိုးရာ", "ပွဲတော်", "ယဉ်ကျေးမှု", "သမိုင်း", "မြန်မာ"]},
    {"id":"mm034", "name": "အင်္ဂလိပ်စာ အခြေခံ", "author": "ဆရာဦးဘုန်း", "description":"အင်္ဂလိပ်စာ စတင်လေ့လာသူများအတွက် အခြေခံသဒ္ဒါနှင့် ဝေါဟာရများ", "category":"ပညာရေး","tags":["ပညာရေး","ဘာသာစကား","အင်္ဂလိပ်စာ","အခြေခံ","သင်ခန်းစာ"]},
    {"id":"eng008", "name": "Psychology of Influence", "author": "Robert C. Cialdini", "description": "Understanding the principles of persuasion and influence in various contexts", "category": "Psychology", "tags": ["psychology", "influence", "persuasion", "behavior", "social science"]},
    {"id":"mm035", "name": "မြန်မာ့ရိုးရာ လက်မှုပညာ", "author": "ဒေါ်နန်း", "description": "မြန်မာ့ရိုးရာ လက်မှုပညာအမျိုးမျိုးနှင့် ပြုလုပ်နည်းအဆင့်ဆင့်", "category": "အနုပညာ", "tags": ["အနုပညာ", "လက်မှုပညာ", "ရိုးရာ", "ပြုလုပ်နည်း", "မြန်မာ"]},
    {"id":"mm036", "name": "ကလေးများအတွက် ကဗျာ", "author": "မောင်စိန်", "description": "ကလေးငယ်များအတွက် ရိုးရှင်းလွယ်ကူသော ကဗျာတိုများ စုစည်းမှု", "category": "ကလေးစာပေ", "tags": ["ကလေး", "ကဗျာ", "စာပေ", "ရိုးရှင်း", "ပျော်စရာ"]},
    {"id":"eng009", "name": "Financial Freedom", "author": "Grant Sabatier", "description": "A guide to achieving financial independence and retiring early", "category": "Finance", "tags": ["finance", "money", "investing", "financial independence", "personal finance"]},
    {"id":"mm037", "name": "မြန်မာ့သဘင်အနုပညာ", "author": "ဦးဖိုးစိန်", "description": "မြန်မာ့ဇာတ်သဘင်အနုပညာ၏ သမိုင်းနှင့် တင်ဆက်မှုပုံစံများ", "category": "အနုပညာ", "tags": ["အနုပညာ", "သဘင်", "ဇာတ်", "သမိုင်း", "မြန်မာ"]},
    {"id":"mm038", "name": "ချက်ပြုတ်နည်း အစုံ", "author": "ဒေါ်ခင်လှိုင်", "description": "မြန်မာ့ရိုးရာနှင့် ခေတ်သစ် ချက်ပြုတ်နည်းအမျိုးမျိုး စုစည်းမှု", "category": "အစားအသောက်", "tags": ["အစားအသောက်", "ချက်ပြုတ်နည်း", "ရိုးရာ", "ခေတ်သစ်", "မြန်မာ"]},
    {"id":"eng010", "name": "The History of the World", "author": "J.M. Roberts", "description": "A comprehensive overview of human history from prehistory to the modern era", "category": "History", "tags": ["history", "world history", "humanity", "civilization", "overview"]},
    {"id":"mm039", "name": "မြန်မာ့ပန်းပုလက်ရာ", "author": "ဦးအောင်ကျော်", "description": "မြန်မာ့ရိုးရာ ပန်းပုလက်ရာများ၏ သမိုင်းနှင့် ပြုလုပ်နည်းများ", "category": "အနုပညာ", "tags": ["အနုပညာ", "ပန်းပု", "ရိုးရာ", "လက်ရာ", "မြန်မာ"]},
    {"id":"mm040", "name": "ကလေးများအတွက် ပုံဆွဲနည်း", "author": "ဆရာဦးကြည်", "description": "ကလေးငယ်များ လွယ်ကူစွာ ပုံဆွဲတတ်ရန် အခြေခံနည်းလမ်းများ", "category": "ကလေးပညာ", "tags": ["ကလေး", "ပုံဆွဲ", "ပညာရေး", "အခြေခံ", "နည်းလမ်း"]},
    {"id":"eng011", "name": "Introduction to Photography", "author": "Ansel Adams", "description": "Basic principles and techniques of photography for beginners", "category": "Art", "tags": ["art", "photography", "techniques", "beginners", "visual arts"]},
    {"id":"mm041", "name": "မြန်မာ့ရိုးရာ အဝတ်အထည်", "author": "ဒေါ်မြမြ", "description": "မြန်မာ့ရိုးရာ အဝတ်အထည်များ၏ သမိုင်းနှင့် ဒီဇိုင်းများ", "category": "ရိုးရာယဉ်ကျေးမှု", "tags": ["ရိုးရာ", "အဝတ်အထည်", "ယဉ်ကျေးမှု", "သမိုင်း", "ဒီဇိုင်း"]},
    {"id":"mm042", "name": "စိတ်ဖိစီးမှု လျှော့ချနည်း", "author": "ဒေါက်တာမောင်မောင်", "description": "စိတ်ဖိစီးမှုကို ထိရောက်စွာ လျှော့ချနိုင်သော နည်းလမ်းများနှင့် လေ့ကျင့်ခန်းများ", "category": "ကျန်းမာရေး", "tags": ["ကျန်းမာရေး", "စိတ်ဖိစီးမှု", "လျှော့ချ", "နည်းလမ်း", "လေ့ကျင့်ခန်း"]},
    {"id":"eng012", "name": "The Science of Sleep", "author": "Matthew Walker", "description": "Understanding the importance of sleep and how to improve your sleep habits", "category": "Health", "tags": ["health", "sleep", "science", "well-being", "habit"]},
     {"id":"mm043", "name": "မြန်မာ့ကျေးလက်ဘဝ", "author": "ဦးတင်", "description": "မြန်မာ့ကျေးလက်နေ ပြည်သူများ၏ ရိုးရှင်းသော ဘဝပုံစံနှင့် ဓလေ့ထုံးတမ်းများ", "category": "လူမှုရေး", "tags": ["လူမှုရေး", "ကျေးလက်", "ဘဝ", "ရိုးရှင်း", "ဓလေ့ထုံးတမ်း"]},
    {"id":"mm044", "name": "ကလေးများအတွက် ပုံပြင်တိုများ", "author": "အန်တီကြည်", "description": "ကလေးငယ်များ နှစ်သက်သော ပုံပြင်တိုများနှင့် သင်ခန်းစာများ", "category": "ကလေးစာပေ", "tags": ["ကလေး", "ပုံပြင်", "စာပေ", "သင်ခန်းစာ", "ပျော်စရာ"]},
    {"id":"eng013", "name": "Machine Learning for Everyone", "author": "Pedro Domingos", "description": "Explaining the core concepts of machine learning in an accessible way", "category": "Technology", "tags": ["technology", "machine learning", "AI", "education", "concepts"]},
    {"id":"mm045", "name": "မြန်မာ့ရိုးရာ အိုးလုပ်ငန်း", "author": "ဦးကျော်", "description": "မြန်မာ့ရိုးရာ အိုးလုပ်ငန်း၏ သမိုင်းနှင့် ပြုလုပ်နည်းအဆင့်ဆင့်", "category": "လက်မှုပညာ", "tags": ["လက်မှုပညာ", "အိုးလုပ်ငန်း", "ရိုးရာ", "ပြုလုပ်နည်း", "မြန်မာ"]},
    {"id":"mm046", "name": "အွန်လိုင်းစီးပွားရေး စတင်ခြင်း", "author": "မောင်မင်း", "description": "အွန်လိုင်းပေါ်တွင် စီးပွားရေးလုပ်ငန်း စတင်ရန် လိုအပ်သော ဗဟုသုတများ", "category": "စီးပွားရေး", "tags": ["စီးပွားရေး", "အွန်လိုင်း", "လုပ်ငန်း", "စတင်ခြင်း", "လမ်းညွှန်"]},
    {"id":"eng014", "name": "The Subtle Art of Not Giving a F*ck", "author": "Mark Manson", "description": "A counterintuitive approach to living a good life", "category": "Self-Help", "tags": ["self-help", "life", "philosophy", "happiness", "personal development"]},
    {"id":"mm047", "name": "မြန်မာ့ကျောက်မျက်ရတနာ", "author": "ဦးစိန်", "description": "မြန်မာနိုင်ငံထွက် ကျောက်မျက်ရတနာအမျိုးမျိုးနှင့် ၄င်းတို့၏ တန်ဖိုး", "category": "ဗဟုသုတ", "tags": ["ဗဟုသုတ", "ကျောက်မျက်ရတနာ", "မြန်မာ", "တန်ဖိုး", "သဘာဝ"]},
    {"id":"mm048", "name": "ကလေးများအတွက် သီချင်းများ", "author": "ဒေါ်မြ", "description": "ကလေးငယ်များ သီဆိုရန် သင့်တော်သော သီချင်းများနှင့် တေးသွားများ", "category": "ကလေးပညာ", "tags": ["ကလေး", "သီချင်း", "တေးဂီတ", "ပညာရေး", "ပျော်စရာ"]},
    {"id":"eng015", "name": "Clean Code", "author": "Robert C. Martin", "description": "A handbook of agile software craftsmanship", "category": "Technology", "tags": ["technology", "programming", "software development", "best practices", "coding"]},
    {"id":"mm049", "name": "မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ", "author": "ဦးမောင်", "description": "မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ၏ သမိုင်းနှင့် ကစားနည်းများ", "category": "အားကစား", "tags": ["အားကစား", "ရိုးရာ", "စားပွဲတင်", "ကစားနည်း", "မြန်မာ"]},
    {"id":"mm050", "name": "ပတ်ဝန်းကျင်ထိန်းသိမ်းရေး", "author": "ဒေါက်တာစိုး", "description": "ပတ်ဝန်းကျင်ထိန်းသိမ်းရေး၏ အရေးပါပုံနှင့် လက်တွေ့လုပ်ဆောင်နိုင်သော နည်းလမ်းများ", "category": "သဘာဝပတ်ဝန်းကျင်", "tags": ["သဘာဝပတ်ဝန်းကျင်", "ထိန်းသိမ်းရေး", "ကာကွယ်ရေး", "နည်းလမ်း", "အရေးပါပုံ"]},
    {"id":"eng016", "name": "Thinking, Fast and Slow", "author": "Daniel Kahneman", "description": "Exploring the two systems that drive the way we think", "category": "Psychology", "tags": ["psychology", "thinking", "behavioral economics", "cognitive science", "decision making"]},
    {"id":"mm051", "name": "မြန်မာ့ကျေးလက်ဒေသ ဖွံ့ဖြိုးတိုးတက်ရေး", "author": "ဦးလှိုင်", "description": "မြန်မာ့ကျေးလက်ဒေသများ၏ စီးပွားရေးနှင့် လူမှုဘဝ ဖွံ့ဖြိုးတိုးတက်ရေးအတွက် နည်းလမ်းများ", "category": "လူမှုရေး", "tags": ["လူမှုရေး", "ကျေးလက်", "ဖွံ့ဖြိုးတိုးတက်ရေး", "စီးပွားရေး", "လူမှုဘဝ"]},
    {"id":"eng017", "name": "The Future of Artificial Intelligence", "author": "Kai-Fu Lee", "description": "How AI will transform our lives and the world", "category": "Technology", "tags": ["AI", "future", "technology", "impact", "society"]},
    {"id":"mm052", "name": "ဗုဒ္ဓဘာသာ ယဉ်ကျေးမှု", "author": "ဒေါက်တာသန်းဦး", "description": "မြန်မာ့ဗုဒ္ဓဘာသာ ယဉ်ကျေးမှု၏ အနှစ်သာရနှင့် ဓလေ့ထုံးတမ်းများ", "category": "ဘာသာရေး", "tags": ["ဘာသာရေး", "ဗုဒ္ဓဘာသာ", "ယဉ်ကျေးမှု", "ဓလေ့ထုံးတမ်း", "မြန်မာ"]},
    {"id":"eng018", "name": "Cybersecurity Essentials", "author": "Bruce Schneier", "description": "Fundamental principles and practices for protecting digital information", "category": "Technology", "tags": ["technology", "cybersecurity", "security", "information", "principles"]},
    {"id":"mm053", "name": "မြန်မာ့စိုက်ပျိုးရေး နည်းပညာ", "author": "ဦးသိန်း", "description": "ခေတ်မီ မြန်မာ့စိုက်ပျိုးရေး နည်းပညာများနှင့် တောင်သူလယ်သမားများအတွက် လမ်းညွှန်", "category": "စိုက်ပျိုးရေး", "tags": ["စိုက်ပျိုးရေး", "နည်းပညာ", "ခေတ်မီ", "တောင်သူလယ်သမား", "လမ်းညွှန်"]},
    {"id":"eng019", "name": "The Psychology of Money", "author": "Morgan Housel", "description": "Timeless lessons on wealth, greed, and happiness", "category": "Finance", "tags": ["finance", "money", "psychology", "wealth", "happiness"]},
    {"id":"mm054", "name": "ကလေးများအတွက် သိပ္ပံ", "author": "ဆရာမိုး", "description": "ကလေးငယ်များ စိတ်ဝင်စားမည့် ရိုးရှင်းသော သိပ္ပံသင်ခန်းစာများ", "category": "ကလေးပညာ", "tags": ["ကလေး", "သိပ္ပံ", "ပညာရေး", "သင်ခန်းစာ", "ရိုးရှင်း"]},
    {"id":"eng020", "name": "Startup Nation", "author": "Dan Senor and Saul Singer", "description": "The story of Israel's economic miracle", "category": "Business", "tags": ["business", "startup", "innovation", "economics", "Israel"]}
  ],
  "queries": [
    {"query":"မြန်မာဘုရင်တွေအကြောင်း ရေးထားတဲ့စာအုပ် ရှာချင်တယ်","expected":"မဟာရာဇာဝင်"},
    {"query":"လွတ်လပ်ရေးတိုက်ပွဲဝင်ခဲ့တဲ့ ရဲဘော်တွေရဲ့အကြောင်း ဝတ္ထုတစ်အုပ်လောက် လိုချင်တယ်","expected":"ထွန်းကားသော ရွှေဂုဏ်ရည်"},
    {"query":"ကုလားမြေဒေသရဲ့ ရိုးရာဓလေ့တွေ လူမှုဘဝတွေအကြောင်း ဖတ်ချင်တယ်","expected":"ကုလားမြေ ဧရိယာများ"},
    {"query":"ရန်ကုန်မှာနေတဲ့ လူငယ်တွေရဲ့ အချစ်နဲ့ ရုန်းကန်မှုအကြောင်း ဝတ္ထု","expected":"ရန်ကုန်မြို့ မုန်းညှင်းကွင်း"},
    {"query":"နယ်က လယ်သမားတွေ မိုးရာသီမှာ ဘယ်လိုဒုက္ခရောက်လဲဆိုတဲ့အကြောင်း","expected":"မိုးရာသီ နောက်လိုက်"},
    {"query":"တက္ကသိုလ်ကျောင်းသားဘဝအကြောင်း ရေးထားတဲ့စာပေ","expected":"တက္ကသိုလ်နေ့များ"},
    {"query":"ခေတ်ပေါ်မြန်မာကဗျာကောင်းလေးတွေ စုစည်းထားတာ ရှိလား","expected":"မြန်မာ့ကဗျာ ရွေးချယ်စု"},
    {"query":"မြန်မာ့ရိုးရာဂီတနဲ့ ခေတ်သစ်ဂီတ ပေါင်းစပ်ထားတဲ့အကြောင်း","expected":"ရွှေမိန်းကလေး ဂီတကမ္ဘာ"},
    {"query":"ဗုဒ္ဓဘာသာရဲ့ အခြေခံတရားတွေကို လမ်းညွှန်ပေးတဲ့စာအုပ်","expected":"ဗုဒ္ဓဓမ္မ လမ်းညွှန်"},
    {"query":"မေတ္တာ၊ ကရုဏာ၊ မုဒိတာ၊ ဥပေက္ခာ အကြောင်း လွယ်လွယ်ကူကူရှင်းပြထားတာ","expected":"မေတ္တာ သုခုမလေး"},
    {"query":"သိပ္ပံပညာရပ်တွေကို မြန်မာလို လွယ်လွယ်နားလည်အောင် ရေးထားတဲ့စာအုပ်","expected":"ခေတ်မီသိပ္ပံနည်းကျ စာကြည့်တိုက်"},
    {"query":"မြန်မာလူငယ်တွေ ကျန်းမာရေးနဲ့ စိတ်ဓာတ်ကောင်းအောင် ဘယ်လိုနေထိုင်ရမလဲ လမ်းညွှန်","expected":"မြန်မာ့သုခုမငယ် ခံ့မြတ်လမ်းစဉ်"},
    {"query":"ကွန်ပျူတာမှာ မြန်မာစာ ဘယ်လိုရိုက်ရလဲ ဘယ်လိုသုံးရလဲ သင်ခန်းစာ","expected":"ကမ္ပျူတာသုံး မြန်မာစာ"},
    {"query":"ဗိုလ်ချုပ်အောင်ဆန်းရဲ့ လွတ်လပ်ရေးရဖို့ ကြိုးပမ်းမှုအကြောင်း","expected":"အောင်ဆန်းနှင့် လွတ်လပ်ရေး"},
    {"query":"ပုဂံခေတ်က ကျောက်စာတွေကနေ ဘာတွေသိရလဲ","expected":"ပုဂံခေတ် ကျောက်စာများ"},
    {"query":"မြန်မာနိုင်ငံရဲ့ စီးပွားရေးအခြေအနေနဲ့ ခေတ်မီစီးပွားရေးလုပ်နည်း","expected":"မြန်မာ့စီးပွားရေး ခေတ်စားလမ်း"},
    {"query":"အွန်လိုင်းကနေ စီးပွားရေးစလုပ်ချင်တယ် လိုအပ်တာတွေ သိချင်တယ်","expected":"အွန်လိုင်းစီးပွားရေး စတင်ခြင်း"},
    {"query":"ကလေးတွေအတွက် စွန့်စားခန်းပုံပြင်လေးတွေ ရှာနေတယ်","expected":"သုံးပေါင်းရဲဘော်"},
    {"query":"ကလေးငယ်တွေအတွက် မနက်ပိုင်းကနေ ညနေပိုင်းအထိ ဘာလုပ်သင့်လဲ လမ်းညွှန်","expected":"ညွန့်ညွန့်ကလေး နံနက်ပိုင်း"},
    {"query":"ကမ္ဘာ့ရာသီဥတု ဘာလို့ပြောင်းလဲနေတာလဲ မြန်မာနိုင်ငံမှာ ဘာတွေဖြစ်လာလဲ","expected":"ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမှု"},
    {"query":"မြန်မာ့ရိုးရာအစားအသောက်တွေရဲ့ နည်းလမ်းတွေ လုပ်ပုံလုပ်နည်းတွေ သိချင်တယ်","expected":"မြန်မာ့ရိုးရာ အစားအသောက်"},
    {"query":"Modern digital marketing strategies and social media techniques","expected":"Digital Marketing Revolution"},
    {"query":"A novel about romance in modern Yangon","expected":"Love in Yangon Streets"},
    {"query":"မြန်မာ့သမိုင်းထဲက စိတ်ဝင်စားဖို့ကောင်းတဲ့ ပုံပြင်တွေ ဒဏ္ဍာရီတွေ","expected":"မြန်မာ့သမိုင်းပုံပြင်များ"},
    {"query":"Contemporary Myanmar paintings and artists","expected":"ခေတ်သစ်ပန်းချီလက်ရာများ"},
    {"query":"Strategies for effective negotiation in business","expected":"The Art of Negotiation"},
    {"query":"လူ့စိတ်နဲ့ စိတ်ပညာရဲ့ အခြေခံသဘောတရားတွေအကြောင်း","expected":"စိတ်ပညာ အခြေခံ"},
    {"query":"မြန်မာ့ရိုးရာတေးဂီတရဲ့ သမိုင်းနဲ့ တူရိယာတွေအကြောင်း","expected":"မြန်မာ့ရိုးရာ တေးဂီတ"},
    {"query":"ကလေးငယ်တွေကို ပုံပြင်စိတ်ဝင်စားအောင် ဘယ်လိုပြောပြမလဲ","expected":"ကလေးများအတွက် ပုံပြောနည်း"},
    {"query":"Introduction to Artificial Intelligence and machine learning concepts","expected":"Introduction to Artificial Intelligence"},
    {"query":"မြန်မာ့ရှေးဟောင်းဗိသုကာလက်ရာတွေရဲ့ အနုပညာနဲ့ သမိုင်းတန်ဖိုး","expected":"မြန်မာ့ရိုးရာ ဗိသုကာလက်ရာများ"},
    {"query":"How to start your own business, basic knowledge and management","expected":"ကိုယ်ပိုင်လုပ်ငန်း စတင်ခြင်း"},
    {"query":"Understanding how habits work and how to change them","expected":"The Power of Habit"},
    {"query":"မြန်မာနိုင်ငံက ဆေးဖက်ဝင်အပင်တွေနဲ့ သူတို့ရဲ့ အကျိုးကျေးဇူးတွေ","expected":"မြန်မာ့ဆေးဖက်ဝင်အပင်များ"},
    {"query":"Traveling to Bagan, a travelogue about historical places","expected":"ခရီးသွားမှတ်တမ်း - ပုဂံ"},
    {"query":"Beginner's guide to data science concepts and tools","expected":"Data Science for Beginners"},
    {"query":"မြန်မာ့ရိုးရာအားကစားနည်းတွေရဲ့ သမိုင်းနဲ့ ကစားနည်း","expected":"မြန်မာ့ရိုးရာ အားကစား"},
    {"query":"How to grow flowers in your garden, basic techniques","expected":"ပန်းဥယျာဉ်စိုက်ပျိုးနည်း"},
    {"query":"Building successful startups using continuous innovation","expected":"The Lean Startup"},
    {"query":"မြန်မာနိုင်ငံတစ်ဝှမ်းက ထင်ရှားတဲ့ ရိုးရာပွဲတော်တွေအကြောင်း","expected":"မြန်မာ့ရိုးရာ ပွဲတော်များ"},
    {"query":"Basic English grammar and vocabulary for beginners","expected":"အင်္ဂလိပ်စာ အခြေခံ"},
    {"query":"Principles of persuasion and influence","expected":"Psychology of Influence"},
    {"query":"မြန်မာ့ရိုးရာလက်မှုပညာအမျိုးမျိုးနဲ့ လုပ်နည်းလုပ်ဟန်","expected":"မြန်မာ့ရိုးရာ လက်မှုပညာ"},
    {"query":"Simple poems for young children","expected":"ကလေးများအတွက် ကဗျာ"},
    {"query":"Guide to financial independence and retiring early","expected":"Financial Freedom"},
    {"query":"မြန်မာ့ဇာတ်သဘင်အနုပညာရဲ့ သမိုင်းနဲ့ တင်ဆက်မှုပုံစံ","expected":"မြန်မာ့သဘင်အနုပညာ"},
    {"query":"Collection of Myanmar traditional and modern recipes","expected":"ချက်ပြုတ်နည်း အစုံ"},
    {"query":"Comprehensive overview of world history","expected":"The History of the World"},
    {"query":"မြန်မာ့ရိုးရာပန်းပုလက်ရာတွေရဲ့ သမိုင်းနဲ့ ပြုလုပ်နည်း","expected":"မြန်မာ့ပန်းပုလက်ရာ"},
    {"query":"Basic drawing techniques for children","expected":"ကလေးများအတွက် ပုံဆွဲနည်း"},
    {"query":"Photography basics and techniques for beginners","expected":"Introduction to Photography"},
    {"query":"မြန်မာ့ရိုးရာအဝတ်အထည်တွေရဲ့ သမိုင်းနဲ့ ဒီဇိုင်း","expected":"မြန်မာ့ရိုးရာ အဝတ်အထည်"},
    {"query":"Effective ways to reduce stress and exercises","expected":"စိတ်ဖိစီးမှု လျှော့ချနည်း"},
    {"query":"Understanding the importance of sleep and improving habits","expected":"The Science of Sleep"},
    {"query":"မြန်မာ့ကျေးလက်နေလူတွေရဲ့ ရိုးရှင်းတဲ့ဘဝပုံစံ","expected":"မြန်မာ့ကျေးလက်ဘဝ"},
    {"query":"Short and enjoyable stories for children","expected":"ကလေးများအတွက် ပုံပြင်တိုများ"},
    {"query":"Core concepts of machine learning explained simply","expected":"Machine Learning for Everyone"},
    {"query":"မြန်မာ့ရိုးရာအိုးလုပ်ငန်းရဲ့ သမိုင်းနဲ့ လုပ်နည်း","expected":"မြန်မာ့ရိုးရာ အိုးလုပ်ငန်း"},
    {"query":"Starting an online business, necessary knowledge","expected":"အွန်လိုင်းစီးပွားရေး စတင်ခြင်း"},
    {"query":"A counterintuitive approach to living a good life","expected":"The Subtle Art of Not Giving a F*ck"},
    {"query":"မြန်မာနိုင်ငံကထွက်တဲ့ ကျောက်မျက်ရတနာအမျိုးမျိုးနဲ့ တန်ဖိုး","expected":"မြန်မာ့ကျောက်မျက်ရတနာ"},
    {"query":"Songs suitable for young children to sing","expected":"ကလေးများအတွက် သီချင်းများ"},
    {"query":"Handbook on best practices for writing clean code","expected":"Clean Code"},
    {"query":"မြန်မာ့ရိုးရာစားပွဲတင်ကစားနည်းတွေရဲ့ သမိုင်းနဲ့ ကစားနည်း","expected":"မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ"},
    {"query":"Importance of environmental conservation and practical methods","expected":"ပတ်ဝန်းကျင်ထိန်းသိမ်းရေး"},
    {"query":"Summary of the book Thinking, Fast and Slow","expected":"Thinking, Fast and Slow"},
    {"query":"ကျေးလက်ဒေသဖွံ့ဖြိုးတိုးတက်ရေးအတွက် ဘာတွေလုပ်သင့်လဲ","expected":"မြန်မာ့ကျေးလက်ဒေသ ဖွံ့ဖြိုးတိုးတက်ရေး"},
    {"query":"How AI will change the world in the future","expected":"The Future of Artificial Intelligence"},
    {"query":"မြန်မာ့ဗုဒ္ဓဘာသာရဲ့ အနှစ်သာရနဲ့ ဓလေ့ထုံးတမ်းတွေ","expected":"ဗုဒ္ဓဘာသာ ယဉ်ကျေးမှု"},
    {"query":"Basic principles for protecting digital information","expected":"Cybersecurity Essentials"},
    {"query":"ခေတ်မီစိုက်ပျိုးရေးနည်းပညာတွေအကြောင်း တောင်သူတွေအတွက်လမ်းညွှန်","expected":"မြန်မာ့စိုက်ပျိုးရေး နည်းပညာ"},
    {"query":"Lessons on wealth, greed, and happiness related to money","expected":"The Psychology of Money"},
    {"query":"ကလေးတွေအတွက် လွယ်ကူတဲ့သိပ္ပံသင်ခန်းစာတွေ","expected":"ကလေးများအတွက် သိပ္ပံ"},
    {"query":"The economic success story of Israel as a startup hub","expected":"Startup Nation"}
  ]
}

books = data["books"]; queries = data["queries"]
book_names = [b["name"] for b in books]


In [6]:
# ========= 2) Text building + Myanmar-friendly tokenizer =========
def build_doc_text(b):
    return " ".join([b["name"], b["author"], b["description"], b["category"], " ".join(b.get("tags", []))])


ZWS_RE = re.compile(r"[\u200B-\u200D\uFEFF\u2060]")

def normalize_mm(s: str) -> str:
    # NFC + strip zero-width + collapse spaces
    s = unicodedata.normalize("NFC", s)
    s = ZWS_RE.sub("", s)
    return s

def mm_tokenize(text: str, min_n: int = 2, max_n: int = 3) -> List[str]:
    # whitespace tokens + Myanmar character n-grams (2–3 chars) to avoid needing a segmenter
    toks = text.strip().split()
    s = "".join(normalize_mm(text).split())
    grams = []
    L = len(s)
    for n in range(min_n, max_n + 1):
        if L >= n:
            grams.extend([s[i:i+n] for i in range(L - n + 1)])
    return toks + grams

docs = [build_doc_text(b) for b in books]

In [7]:
# ========= 3) Indexes =========
class BM25Index:
    def __init__(self, docs: List[str]):
        tokenized = [mm_tokenize(d) for d in docs]
        self.bm25 = BM25Okapi(tokenized)
    def search(self, q: str, topk: int = 50):
        toks = mm_tokenize(q)
        scores = self.bm25.get_scores(toks)
        idx = np.argsort(-scores)[:topk]
        return idx, scores[idx]

class DenseEncoder:
    def __init__(self, model_id: str, device, e5_prefix=False):
        self.model = SentenceTransformer(model_id, device=str(device))
        self.e5_prefix = e5_prefix
    def encode_docs(self, docs: List[str], batch_size=DENSE_BATCH) -> np.ndarray:
        xs = ["passage: " + d if self.e5_prefix else d for d in docs]
        emb = self.model.encode(xs, batch_size=batch_size, normalize_embeddings=True, show_progress_bar=True)
        return emb.astype(np.float32)
    def encode_queries(self, queries: List[str]) -> np.ndarray:
        xs = ["query: " + q if self.e5_prefix else q for q in queries]
        emb = self.model.encode(xs, batch_size=64, normalize_embeddings=True, show_progress_bar=False)
        return emb.astype(np.float32)

class CrossEncoderReranker:
    def __init__(self, model_id: str, device):
        self.tok = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)
        self.model.eval(); self.device = device
    @torch.no_grad()
    def score(self, query: str, passages: List[str], batch_size=RERANK_BATCH) -> np.ndarray:
        out = []
        for i in range(0, len(passages), batch_size):
            batch = passages[i:i+batch_size]
            enc = self.tok([query]*len(batch), batch, padding=True, truncation=True,
                           max_length=512, return_tensors="pt").to(self.device)
            s = self.model(**enc).logits.squeeze(-1).detach().cpu().numpy()
            out.append(s)
        return np.concatenate(out, axis=0)

def dense_search(qe: np.ndarray, doc_emb: np.ndarray, topk=50):
    sims = qe @ doc_emb.T  # cosine if normalized=True
    idx = np.argsort(-sims)[:topk]
    return idx, sims[idx]

def rrf_fuse(rank_lists: List[np.ndarray], k: int = 60) -> np.ndarray:
    ids = set()
    for r in rank_lists: ids.update(r.tolist())
    scores = {i: 0.0 for i in ids}
    for r in rank_lists:
        for rank, doc_id in enumerate(r, start=1):
            scores[doc_id] += 1.0 / (k + rank)
    fused = sorted(ids, key=lambda i: -scores[i])
    return np.array(fused, dtype=int)

In [8]:
# ========= 4) Build indexes (with safe fallbacks) =========
print("Building BM25…")
bm25 = BM25Index(docs)

dense_e5 = doc_e5 = None
dense_mpnet = doc_mpnet = None

try:
    print("Loading E5 (intfloat/multilingual-e5-base)…")
    dense_e5 = DenseEncoder("intfloat/multilingual-e5-base", DEV, e5_prefix=True)
    doc_e5 = dense_e5.encode_docs(docs)
    print("E5 ready.")
except Exception as e:
    print("E5 failed:", repr(e))

try:
    print("Loading mpnet (sentence-transformers/paraphrase-multilingual-mpnet-base-v2)…")
    dense_mpnet = DenseEncoder("sentence-transformers/paraphrase-multilingual-mpnet-base-v2", DEV)
    doc_mpnet = dense_mpnet.encode_docs(docs)
    print("mpnet ready.")
except Exception as e:
    print("mpnet failed:", repr(e))

Building BM25…
Loading E5 (intfloat/multilingual-e5-base)…


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

mpnet ready.


In [9]:
# ========= 5) Evaluation =========
@dataclass
class MethodResult:
    method: str
    accuracy: float
    mrr10: float
    recall10: float
    p1: float
    p5: float
    p10: float
    map: float
    avg_top1_score: float
    time_ms: float

def eval_method(pretty: str, search_fn):
    t0_total = time.perf_counter()
    rows = []
    acc = mrr = rec = p1 = p5 = p10 = ap_sum = 0.0
    query_times = []

    for qobj in tqdm(queries, desc=f"Evaluating {pretty}"):
        q, gold = qobj["query"], qobj["expected"]
        t0_q = time.perf_counter()
        idx, scores = search_fn(q)
        query_times.append(time.perf_counter() - t0_q)

        names = [book_names[i] for i in idx]
        top1 = names[0] if len(names) else ""
        relevance = [1 if name == gold else 0 for name in names]

        acc += 1.0 if top1 == gold else 0.0
        # MRR@10
        hit_rank = next((i+1 for i,n in enumerate(names[:10]) if n==gold), None)
        mrr += (1.0 / hit_rank) if hit_rank else 0.0
        # Recall@10
        rec += 1.0 if gold in names[:10] else 0.0
        # Precision@K
        p1  += relevance[0] if relevance else 0.0
        p5  += np.mean(relevance[:5])  if len(relevance) >= 5  else 0.0
        p10 += np.mean(relevance[:10]) if len(relevance) >= 10 else 0.0
        # AP (only 1 relevant per query)
        ap = 0.0
        rel_so_far = 0
        for i, r in enumerate(relevance):
            if r == 1:
                rel_so_far += 1
                ap += rel_so_far / (i + 1)
        ap_sum += ap

        rows.append({
            "query": q,
            "expected": gold,
            "predicted": top1,
            "top1_score": float(scores[0]) if len(scores) else float("-inf"),
            "correct": top1 == gold
        })

    n = len(queries)
    total_time = time.perf_counter() - t0_total
    top1_scores = [r["top1_score"] for r in rows if np.isfinite(r["top1_score"])]
    avg_top1 = float(np.mean(top1_scores)) if top1_scores else float("nan")

    out = MethodResult(
        method=pretty,
        accuracy=100.0 * acc / n,
        mrr10=100.0 * mrr / n,
        recall10=100.0 * rec / n,
        p1=100.0 * p1 / n,
        p5=100.0 * p5 / n,
        p10=100.0 * p10 / n,
        map=100.0 * ap_sum / n,
        avg_top1_score=avg_top1,
        time_ms=1000.0 * total_time
    )
    return out, pd.DataFrame(rows)

results = []
per_method = {}


In [10]:
# Sparse baseline
def bm25_search(q): return bm25.search(q, topk=50)
r, df = eval_method("Sparse (BM25)", bm25_search); results.append(r); per_method[r.method] = df

# Dense E5
if doc_e5 is not None:
    def e5_search(q):
        qe = dense_e5.encode_queries([q])[0]
        return dense_search(qe, doc_e5, topk=50)
    r, df = eval_method("Dense (E5-base)", e5_search); results.append(r); per_method[r.method] = df

# Dense MPNet
if doc_mpnet is not None:
    def mpnet_search(q):
        qe = dense_mpnet.encode_queries([q])[0]
        return dense_search(qe, doc_mpnet, topk=50)
    r, df = eval_method("Dense (mpnet-multilingual)", mpnet_search); results.append(r); per_method[r.method] = df

# Hybrid RRF
if (doc_e5 is not None) or (doc_mpnet is not None):
    use_dense = "E5" if doc_e5 is not None else "mpnet"
    def dense_fn(q):
        if use_dense == "E5":
            qe = dense_e5.encode_queries([q])[0]; return dense_search(qe, doc_e5, topk=50)
        else:
            qe = dense_mpnet.encode_queries([q])[0]; return dense_search(qe, doc_mpnet, topk=50)
    def hybrid_rrf(q):
        idx_bm25, _ = bm25.search(q, topk=50)
        idx_dense, _ = dense_fn(q)
        fused = rrf_fuse([idx_bm25, idx_dense])[:50]
        return fused, np.arange(len(fused))[::-1]
    r, df = eval_method(f"Hybrid RRF (BM25+{use_dense})", hybrid_rrf); results.append(r); per_method[r.method] = df

# Hybrid + Rerank
try:
    if (doc_e5 is not None) or (doc_mpnet is not None):
        print(f"Loading reranker: {RERANK_MODEL} …")
        rerank = CrossEncoderReranker(RERANK_MODEL, DEV)
        def dense_fn(q):
            if doc_e5 is not None:
                qe = dense_e5.encode_queries([q])[0]; return dense_search(qe, doc_e5, topk=50)
            else:
                qe = dense_mpnet.encode_queries([q])[0]; return dense_search(qe, doc_mpnet, topk=50)
        def hybrid_rerank(q):
            idx_bm25, _ = bm25.search(q, topk=50)
            idx_dense, _ = dense_fn(q)
            fused = rrf_fuse([idx_bm25, idx_dense])[:SHORTLIST_K]
            cands = [docs[i] for i in fused]
            scores = rerank.score(q, cands)
            order = np.argsort(-scores)
            idx = fused[order]
            return idx[:50], scores[order][:50]
        r, df = eval_method(f"Hybrid+Rerank ({RERANK_MODEL})", hybrid_rerank); results.append(r); per_method[r.method] = df
    else:
        print("Skip rerank: no dense encoder available.")
except Exception as e:
    print("Reranker failed:", repr(e))

Evaluating Sparse (BM25):   0%|          | 0/74 [00:00<?, ?it/s]

Evaluating Dense (E5-base):   0%|          | 0/74 [00:00<?, ?it/s]

Evaluating Dense (mpnet-multilingual):   0%|          | 0/74 [00:00<?, ?it/s]

Evaluating Hybrid RRF (BM25+E5):   0%|          | 0/74 [00:00<?, ?it/s]

Loading reranker: BAAI/bge-reranker-base …


...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Evaluating Hybrid+Rerank (BAAI/bge-reranker-base):   0%|          | 0/74 [00:00<?, ?it/s]

In [11]:
# ========= 6) Report =========
metrics_df = pd.DataFrame([r.__dict__ for r in results]).sort_values("accuracy", ascending=False).reset_index(drop=True)
display(metrics_df)

def errors_for(method_name, k=10):
    df = per_method[method_name]
    errs = df[~df["correct"]].copy()
    return errs.sort_values("top1_score", ascending=False).head(k).reset_index(drop=True)

best = metrics_df.iloc[0]["method"]
print("Best method:", best)
display(errors_for(best, k=15))

def compare_pair(m1, m2, k=20):
    a = per_method[m1][["query","expected","predicted","correct"]].rename(
        columns={"predicted": f"pred_{m1}", "correct": f"ok_{m1}"})
    b = per_method[m2][["query","predicted","correct"]].rename(
        columns={"predicted": f"pred_{m2}", "correct": f"ok_{m2}"})
    flip = a.merge(b, on="query")
    flip = flip[(flip[f"ok_{m1}"] != flip[f"ok_{m2}"])]
    return flip.head(k).reset_index(drop=True)

if "Dense (E5-base)" in per_method:
    display(compare_pair("Sparse (BM25)", "Dense (E5-base)"))
elif "Dense (mpnet-multilingual)" in per_method:
    display(compare_pair("Sparse (BM25)", "Dense (mpnet-multilingual)"))
else:
    print("No dense model loaded; only BM25 results available.")

,method,accuracy,mrr10,recall10,p1,p5,p10,map,avg_top1_score,time_ms
0,Dense (E5-base),97.297297,98.243243,100.000000,97.297297,20.000000,10.000000,98.243243,0.867403,892.027000
1,Dense (mpnet-multilingual),93.243243,95.720721,98.648649,93.243243,19.729730,9.864865,95.824671,0.830661,829.996066
2,Sparse (BM25),82.432432,82.432432,82.432432,82.432432,16.486486,8.243243,82.595613,179.312969,185.036772
3,Hybrid RRF (BM25+E5),81.081081,83.124196,89.189189,81.081081,17.027027,8.918919,83.447949,49.000000,1037.721590
4,Hybrid+Rerank (BAAI/bge-reranker-base),72.972973,79.189189,89.189189,72.972973,17.027027,8.918919,79.189189,0.197160,4372.034534


Best method: Dense (E5-base)


,query,expected,predicted,top1_score,correct
0,Short and enjoyable stories for children,ကလေးများအတွက် ပုံပြင်တိုများ,ကလေးများအတွက် ကဗျာ,0.870092,False
1,ကလေးတွေအတွက် စွန့်စားခန်းပုံပြင်လေးတွေ ရှာနေတယ်,သုံးပေါင်းရဲဘော်,ကလေးများအတွက် ပုံပြင်တိုများ,0.827430,False


,query,expected,pred_Sparse (BM25),ok_Sparse (BM25),pred_Dense (E5-base),ok_Dense (E5-base)
0,ကလေးတွေအတွက် စွန့်စားခန်းပုံပြင်လေးတွေ ရှာနေတယ်,သုံးပေါင်းရဲဘော်,သုံးပေါင်းရဲဘော်,True,ကလေးများအတွက် ပုံပြင်တိုများ,False
1,Contemporary Myanmar paintings and artists,ခေတ်သစ်ပန်းချီလက်ရာများ,Digital Marketing Revolution,False,ခေတ်သစ်ပန်းချီလက်ရာများ,True
2,"How to start your own business, basic knowledg...",ကိုယ်ပိုင်လုပ်ငန်း စတင်ခြင်း,The Lean Startup,False,ကိုယ်ပိုင်လုပ်ငန်း စတင်ခြင်း,True
3,"Traveling to Bagan, a travelogue about histori...",ခရီးသွားမှတ်တမ်း - ပုဂံ,The Subtle Art of Not Giving a F*ck,False,ခရီးသွားမှတ်တမ်း - ပုဂံ,True
4,"How to grow flowers in your garden, basic tech...",ပန်းဥယျာဉ်စိုက်ပျိုးနည်း,Introduction to Photography,False,ပန်းဥယျာဉ်စိုက်ပျိုးနည်း,True
5,Basic English grammar and vocabulary for begin...,အင်္ဂလိပ်စာ အခြေခံ,Introduction to Photography,False,အင်္ဂလိပ်စာ အခြေခံ,True
6,Simple poems for young children,ကလေးများအတွက် ကဗျာ,Machine Learning for Everyone,False,ကလေးများအတွက် ကဗျာ,True
7,Collection of Myanmar traditional and modern r...,ချက်ပြုတ်နည်း အစုံ,Love in Yangon Streets,False,ချက်ပြုတ်နည်း အစုံ,True
8,Basic drawing techniques for children,ကလေးများအတွက် ပုံဆွဲနည်း,Introduction to Photography,False,ကလေးများအတွက် ပုံဆွဲနည်း,True
9,Effective ways to reduce stress and exercises,စိတ်ဖိစီးမှု လျှော့ချနည်း,The Art of Negotiation,False,စိတ်ဖိစီးမှု လျှော့ချနည်း,True


In [12]:
# ========= 7) Per-method TOP-K result viewer =========
import numpy as np
import pandas as pd

TOPK = 10  # how many results to show per method

# Collect whichever methods are actually available in *your* run
METHODS = [("Sparse (BM25)", bm25_search)]
if 'e5_search' in globals():          METHODS.append(("Dense (E5-base)", e5_search))
if 'mpnet_search' in globals():       METHODS.append(("Dense (mpnet-multilingual)", mpnet_search))
if 'hybrid_rrf' in globals():
    dense_tag = "E5" if ('doc_e5' in globals() and doc_e5 is not None) else "mpnet"
    METHODS.append((f"Hybrid RRF (BM25+{dense_tag})", hybrid_rrf))
if 'hybrid_rerank' in globals():      METHODS.append((f"Hybrid+Rerank ({RERANK_MODEL})", hybrid_rerank))

def run_method(fn, q, topk=TOPK):
    """Return names and scores for a single query."""
    idx, scores = fn(q)
    k = min(topk, len(idx))
    names = [book_names[i] for i in idx[:k]]
    return names, scores[:k]

def inspect_query(q, gold=None, topk=TOPK):
    """Pretty table: top-k per method for one query."""
    rows = []
    for mname, fn in METHODS:
        names, scores = run_method(fn, q, topk)
        # find gold rank (if provided)
        gold_rank = None
        if gold is not None:
            for r, n in enumerate(names, start=1):
                if n == gold:
                    gold_rank = r
                    break
        rows.append({
            "method": mname,
            "topk_titles": " | ".join(names),
            "gold_rank": gold_rank,
            "contains_gold@k": gold_rank is not None if gold is not None else None
        })
    return pd.DataFrame(rows)

# ---- A) Inspect a single ad-hoc query
user_query = "ရန်ကုန်မှာနေတဲ့ လူငယ်တွေရဲ့ အချစ်နဲ့ ရုန်းကန်မှုအကြောင်း ဝတ္ထု"
display(inspect_query(user_query, gold=None, topk=TOPK))

# ---- B) Inspect a sample of 10 dataset queries (change seed/size as you like)
rng = np.random.RandomState(42)
sample_idx = rng.choice(len(queries), size=min(10, len(queries)), replace=False)

all_frames = []
for i in sample_idx:
    q = queries[i]["query"]; gold = queries[i]["expected"]
    dfq = inspect_query(q, gold=gold, topk=TOPK)
    dfq.insert(0, "expected", gold)
    dfq.insert(0, "query", q)
    all_frames.append(dfq)

sample_view = pd.concat(all_frames, ignore_index=True)
display(sample_view)

# ---- C) (optional) Dump FULL top-K for *every* query & method to CSV
def dump_all_topk(topk=TOPK, path="topk_by_method.csv"):
    rows = []
    for qobj in queries:
        q, gold = qobj["query"], qobj["expected"]
        for mname, fn in METHODS:
            idx, scores = fn(q)
            k = min(topk, len(idx))
            for r in range(k):
                rows.append({
                    "query": q,
                    "expected": gold,
                    "method": mname,
                    "rank": r+1,
                    "title": book_names[idx[r]],
                    "score": float(scores[r])  # note: scores are NOT comparable across methods
                })
    df = pd.DataFrame(rows)
    df.to_csv(path, index=False)
    return df

# Uncomment to create the CSV and preview first 30 rows:
# full_df = dump_all_topk(TOPK)
# display(full_df.head(30))


,method,topk_titles,gold_rank,contains_gold@k
0,Sparse (BM25),ရန်ကုန်မြို့ မုန်းညှင်းကွင်း | ထွန်းကားသော ရွှ...,None,None
1,Dense (E5-base),ရန်ကုန်မြို့ မုန်းညှင်းကွင်း | ထွန်းကားသော ရွှ...,None,None
2,Dense (mpnet-multilingual),ရန်ကုန်မြို့ မုန်းညှင်းကွင်း | မြန်မာ့ကျေးလက်ဘ...,None,None
3,Hybrid RRF (BM25+E5),ရန်ကုန်မြို့ မုန်းညှင်းကွင်း | ထွန်းကားသော ရွှ...,None,None
4,Hybrid+Rerank (BAAI/bge-reranker-base),ရန်ကုန်မြို့ မုန်းညှင်းကွင်း | တက္ကသိုလ်နေ့မျာ...,None,None


,query,expected,method,topk_titles,gold_rank,contains_gold@k
0,နယ်က လယ်သမားတွေ မိုးရာသီမှာ ဘယ်လိုဒုက္ခရောက်လဲ...,မိုးရာသီ နောက်လိုက်,Sparse (BM25),မိုးရာသီ နောက်လိုက် | ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမ...,1.0,True
1,နယ်က လယ်သမားတွေ မိုးရာသီမှာ ဘယ်လိုဒုက္ခရောက်လဲ...,မိုးရာသီ နောက်လိုက်,Dense (E5-base),မိုးရာသီ နောက်လိုက် | ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမ...,1.0,True
2,နယ်က လယ်သမားတွေ မိုးရာသီမှာ ဘယ်လိုဒုက္ခရောက်လဲ...,မိုးရာသီ နောက်လိုက်,Dense (mpnet-multilingual),မိုးရာသီ နောက်လိုက် | ပန်းဥယျာဉ်စိုက်ပျိုးနည်း...,1.0,True
3,နယ်က လယ်သမားတွေ မိုးရာသီမှာ ဘယ်လိုဒုက္ခရောက်လဲ...,မိုးရာသီ နောက်လိုက်,Hybrid RRF (BM25+E5),မိုးရာသီ နောက်လိုက် | ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမ...,1.0,True
4,နယ်က လယ်သမားတွေ မိုးရာသီမှာ ဘယ်လိုဒုက္ခရောက်လဲ...,မိုးရာသီ နောက်လိုက်,Hybrid+Rerank (BAAI/bge-reranker-base),မိုးရာသီ နောက်လိုက် | ကမ္ဘာ့ရာသီဥတု ပြောင်းလဲမ...,1.0,True
5,မြန်မာ့ရိုးရာစားပွဲတင်ကစားနည်းတွေရဲ့ သမိုင်းနဲ...,မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ,Sparse (BM25),မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ | မြန်မာ့ရ...,1.0,True
6,မြန်မာ့ရိုးရာစားပွဲတင်ကစားနည်းတွေရဲ့ သမိုင်းနဲ...,မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ,Dense (E5-base),မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ | မြန်မာ့ရ...,1.0,True
7,မြန်မာ့ရိုးရာစားပွဲတင်ကစားနည်းတွေရဲ့ သမိုင်းနဲ...,မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ,Dense (mpnet-multilingual),မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ | မြန်မာ့ရ...,1.0,True
8,မြန်မာ့ရိုးရာစားပွဲတင်ကစားနည်းတွေရဲ့ သမိုင်းနဲ...,မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ,Hybrid RRF (BM25+E5),မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ | မြန်မာ့ရ...,1.0,True
9,မြန်မာ့ရိုးရာစားပွဲတင်ကစားနည်းတွေရဲ့ သမိုင်းနဲ...,မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ,Hybrid+Rerank (BAAI/bge-reranker-base),မြန်မာ့ရိုးရာ စားပွဲတင်ကစားနည်းများ | မြန်မာ့ရ...,1.0,True
